In [1]:
import tropycal.tracks as tracks
import datetime as dt
import numpy as np
import pandas as pd

/Users/cynthiazeng/anaconda3/lib/python3.7/site-packages/tropycal/plot/plot.py:16: UserWarning: Warning: Cartopy is not installed in your python environment. Plotting functions will not work.
  warnings.warn("Warning: Cartopy is not installed in your python environment. Plotting functions will not work.")
/Users/cynthiazeng/anaconda3/lib/python3.7/site-packages/tropycal/tracks/plot.py:24: UserWarning: Warning: Cartopy is not installed in your python environment. Plotting functions will not work.
  warnings.warn("Warning: Cartopy is not installed in your python environment. Plotting functions will not work.")
/Users/cynthiazeng/anaconda3/lib/python3.7/site-packages/tropycal/tornado/dataset.py:21: UserWarning: Warning: Cartopy is not installed in your python environment. Plotting functions will not work.
  warnings.warn(warn_message)
/Users/cynthiazeng/anaconda3/lib/python3.7/site-packages/tropycal/tornado/plot.py:21: UserWarning: Warning: Cartopy is not installed in your python environm

In [2]:
import sys
sys.path.append('../')
from utils import data_processing as proc

In [3]:
#read north atlantic datasets
hurdat= tracks.TrackDataset(basin='both',source='hurdat',include_btk=False)
#example to get forecast for one storm 
storm = hurdat.get_storm(('michael', 2018))
forecast = storm.get_operational_forecasts()
print('all available models', storm.get_operational_forecasts().keys()) 

# I chose models to use:
# #'HWRF','SHIP','NVGM','AEMI','NAM 
model_list = set(['HWRF','SHIP', 'AEMI','CLAP5','EMXI','CMC']).intersection(forecast.keys())

#This website has the explanation of all the models: 
#https://www.nhc.noaa.gov/modelsummary.shtml#:~:text=The%20National%20Hurricane%20Center%20 

--> Starting to read in HURDAT2 data
--> Completed reading in HURDAT2 data (10.51 seconds)
all available models dict_keys(['CARQ', 'NAM', 'AC00', 'AEMN', 'AP01', 'AP02', 'AP03', 'AP04', 'AP05', 'AP06', 'AP07', 'AP08', 'AP09', 'AP10', 'AP11', 'AP12', 'AP13', 'AP14', 'AP15', 'AP16', 'AP17', 'AP18', 'AP19', 'AP20', 'AVNO', 'AVNX', 'CLP5', 'CTCX', 'DSHP', 'GFSO', 'HCCA', 'IVCN', 'IVDR', 'LGEM', 'OCD5', 'PRFV', 'SHF5', 'SHIP', 'TABD', 'TABM', 'TABS', 'TCLP', 'XTRP', 'CMC', 'NGX', 'UKX', 'AEMI', 'AHNI', 'AVNI', 'CEMN', 'CHCI', 'CTCI', 'DSPE', 'EGRR', 'LGME', 'NAMI', 'NEMN', 'RVCN', 'RVCX', 'SHPE', 'TBDE', 'TBME', 'TBSE', 'TVCA', 'TVCE', 'TVCN', 'TVCX', 'TVDG', 'UE00', 'UE01', 'UE02', 'UE03', 'UE04', 'UE05', 'UE06', 'UE07', 'UE08', 'UE09', 'UE10', 'UE11', 'UE12', 'UE13', 'UE14', 'UE15', 'UE16', 'UE17', 'UE18', 'UE19', 'UE20', 'UE21', 'UE22', 'UE23', 'UE24', 'UE25', 'UE26', 'UE27', 'UE28', 'UE29', 'UE30', 'UE31', 'UE32', 'UE33', 'UE34', 'UE35', 'UEMN', 'CEMI', 'CMCI', 'COTC', 'EGRI', 'HMON', '

In [12]:
from utils import data_processing as proc
#testing that the code works 
e[:, :, 1:], d, e = proc.prepare_tabular_data_vision(path="../data/last3years.csv", min_wind=50, min_steps=15,
                  max_steps=120, get_displacement=True, forecast=True, one_hot = False)

joining forecast data
--> Starting to read in HURDAT2 data
--> Completed reading in HURDAT2 data (10.12 seconds)
no forecast for pali 2016
no forecast for victor 2016
no forecast for corentin 2016
no forecast for stan 2016
no forecast for daya 2016
no forecast for uriah 2016
no forecast for tatiana 2016
no forecast for winston 2016
no forecast for yalo 2016
no forecast for not_named 2016
no forecast for emeraude 2016
no forecast for not_named 2016
no forecast for zena 2016
no forecast for fantala 2016
no forecast for amos 2016
no forecast for roanu 2016
no forecast for not_named 2016
no forecast for not_named 2016
no forecast for not_named 2016
no forecast for nepartak 2016
no forecast for abela 2016
no forecast for not_named 2016
no forecast for lupit 2016
no forecast for mirinae 2016
no forecast for nida 2016
no forecast for omais 2016
no forecast for conson 2016
no forecast for not_named 2016
no forecast for chanthu 2016
no forecast for dianmu 2016
no forecast for mindulle 2016
no f

KeyboardInterrupt: 

In [8]:
#function to get forecast of hurricane based on name and year 
def get_forecast(hurdat, name, year, pred=24): #pred: hours prediction 
    try:
        storm = hurdat.get_storm((name, year))
        forecast = storm.get_operational_forecasts()
        #choose models 
        model_list = set(['NAM','AP01','HWRF','SHIP', 'AEMI','CLAP5','EMXI','CMC']).intersection(forecast.keys())
        #create empty df 
        df_out = pd.DataFrame(columns=['datetime'])
        for model in model_list:  
            df_model = pd.DataFrame()
            for time in forecast[model]:    
                df = pd.DataFrame(forecast[model][time])
                temp = df.loc[df['fhr']==pred]
                #select columns
                temp = temp[['lat','lon','vmax','mslp']]
                temp = temp.add_prefix(str(model)+'_'+str(pred)+'_')
                temp['datetime'] = pd.to_datetime(time, format = '%Y%m%d%H')
                df_model = pd.concat([df_model, temp], axis=0)
            df_out = df_out.merge(df_model, on='datetime', how='outer')
        df_out = df_out.sort_values(by='datetime')         
    except:
        print('no forecast for', name, year)
        df_out = pd.DataFrame(columns=['datetime'])
    return df_out

def join_forecast(df, pred=24):
    print('joining forecast data')
    #create a dataframe to store joined data
    df_all= pd.DataFrame()

    # read hurdat data set for both basins: north atlantic and east pacific
    hurdat = tracks.TrackDataset(basin='both',source='hurdat',include_btk=False)

    #get list of storm names and year
    df['NAME'] = df['NAME'].str.lower()
    df['YEAR'] = df['ISO_TIME'].dt.year
    storm_list = df[['SID','NAME','YEAR','BASIN']].drop_duplicates(['SID']) #get list of storm names and year
    storm_list.reset_index(inplace=True, drop=True)

    for i in range(len(storm_list)):
        SID = storm_list['SID'][i]
        name = storm_list['NAME'][i]
        year= storm_list['YEAR'][i]
        basin = storm_list['BASIN'][i]
        
        #get stat data for particular storm
        df_stat = df.loc[df['NAME']==name]
        
        #no forecast for these basins 
        if basin in ['SP','SI','NI','WP']:
            df_all = pd.concat([df_all, df_stat], axis=0)        
        else:  
            #try to get forecast for particular storm
            df_forecast = get_forecast(hurdat, name, year, pred)
            if df_forecast.shape[0]>0:
                #join with dataframe
                df_forecast['NAME']= name
                df_joined = df_stat.merge(df_forecast, how='left', left_on=['ISO_TIME','NAME'], right_on=['datetime','NAME'])
                df_joined.drop('datetime', axis=1, inplace=True)
                df_all = pd.concat([df_all, df_joined], axis=0)
            else:
                df_all = pd.concat([df_all, df_stat], axis=0)        
    #drop added columns
    df_all.drop(['NAME','YEAR'], inplace=True, axis=1)
    df_all.reset_index(inplace=True, drop=True)
    print("The dataframe of storms with forecast has been created.")
    return df_all

In [10]:
#testing it works with the prepare_tabular_vision code 
min_wind=50
min_steps=15
max_steps=120 
get_displacement=True 
one_hot = False
#documentation on data https://www.ncdc.noaa.gov/ibtracs/pdf/IBTrACS_v04_column_documentation.pdf 
# path="../data/since1980.csv"
path="../data/last3years.csv"
data = pd.read_csv(path)
data.drop(0, axis=0, inplace=True) #drop secondary column names
# select interesting columns
df0 = data[['SID','NAME', 'BASIN','ISO_TIME', 'LAT', 'LON', 'WMO_WIND', 'WMO_PRES', 'DIST2LAND', 'STORM_SPEED', 'STORM_DIR']]
# transform data from String to numeric
df0 = proc.numeric_data(df0)
# smooth cos & sign of day
df0 = proc.smooth_features(df0)
# # add wind category
df0['wind_category'] = df0.apply(lambda x: proc.sust_wind_to_cat_val(x['WMO_WIND']), axis=1)
# df0 = df0.iloc[-10000:-7000]

In [57]:
df0.shape
df1.shape

(73, 18)

In [11]:
df1 = df0.iloc[15700:16000]
df1 = join_forecast(df1, pred=24)
df2 = proc.add_one_hot(df1, df1['BASIN'],'_basin')
df2.head()

joining forecast data
--> Starting to read in HURDAT2 data
--> Completed reading in HURDAT2 data (6.44 seconds)
The dataframe of storms with forecast has been created.
one-hot added for  BASIN


,SID,BASIN,ISO_TIME,LAT,LON,WMO_WIND,WMO_PRES,DIST2LAND,STORM_SPEED,cos_day,...,NAM_24_lon,NAM_24_vmax,NAM_24_mslp,CMC_24_lat,CMC_24_lon,CMC_24_vmax,CMC_24_mslp,_basin_EP,_basin_NI,_basin_WP
0,2018273N12259,EP,2018-10-11 21:00:00,23.4571,-117.099,55.0,992.0,494.0,20.0,0.982126,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,0
1,2018273N12259,EP,2018-10-12 00:00:00,24.1000,-116.300,55.0,992.0,395.0,19.0,0.978740,...,-111.2,26.0,1003.0,29.6,-111.3,25.0,1001.0,1,0,0
2,2018273N12259,EP,2018-10-12 03:00:00,24.6995,-115.506,55.0,992.0,293.0,19.0,0.978740,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,0
3,2018273N12259,EP,2018-10-12 06:00:00,25.3000,-114.700,55.0,992.0,191.0,19.0,0.978740,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,0
4,2018273N12259,EP,2018-10-12 09:00:00,25.9270,-113.864,50.0,993.0,94.0,20.0,0.978740,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0,0


In [14]:
df1

,SID,BASIN,ISO_TIME,LAT,LON,WMO_WIND,WMO_PRES,DIST2LAND,STORM_SPEED,cos_day,...,HWRF_24_vmax,HWRF_24_mslp,NAM_24_lat,NAM_24_lon,NAM_24_vmax,NAM_24_mslp,CMC_24_lat,CMC_24_lon,CMC_24_vmax,CMC_24_mslp
0,2018273N12259,EP,2018-10-11 21:00:00,23.4571,-117.099,55.000000,992.000000,494.0,20.0,0.982126,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2018273N12259,EP,2018-10-12 00:00:00,24.1000,-116.300,55.000000,992.000000,395.0,19.0,0.978740,...,37.0,1004.0,29.9,-111.2,26.0,1003.0,29.6,-111.3,25.0,1001.0
2,2018273N12259,EP,2018-10-12 03:00:00,24.6995,-115.506,55.000000,992.000000,293.0,19.0,0.978740,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2018273N12259,EP,2018-10-12 06:00:00,25.3000,-114.700,55.000000,992.000000,191.0,19.0,0.978740,...,35.0,1006.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2018273N12259,EP,2018-10-12 09:00:00,25.9270,-113.864,50.000000,993.000000,94.0,20.0,0.978740,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,2018295N08158,WP,2018-10-22 21:00:00,11.1999,152.143,43.750000,992.000000,1563.0,12.0,0.929141,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
296,2018295N08158,WP,2018-10-23 00:00:00,11.5000,151.600,43.645833,992.166667,1592.0,11.0,0.922640,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
297,2018295N08158,WP,2018-10-23 03:00:00,11.6799,151.092,43.541667,992.333333,1599.0,10.0,0.922640,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
298,2018295N08158,WP,2018-10-23 06:00:00,11.8000,150.600,43.437500,992.500000,1597.0,10.0,0.922640,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
# df0.loc[df0['NAME']=='michael']['BASIN']
# df1
# read hurdat data set for both basins: north atlantic and east pacific
# hurdat = tracks.TrackDataset(basin='both',source='hurdat',include_btk=False)
get_forecast(hurdat, 'michael',2018, 24)
# pred=24
# storm = hurdat.get_storm(('michael', 2018))
# forecast = storm.get_operational_forecasts()
# #choose models 
# model_list = set(['NAM','AP01','HWRF','SHIP', 'AEMI','CLAP5','EMXI','CMC']).intersection(forecast.keys())
# #create empty df 
# df_out = pd.DataFrame(columns=['datetime'])
# for model in model_list:  
#     df_model = pd.DataFrame()
#     for time in forecast[model]:    
#         df = pd.DataFrame(forecast[model][time])
#         temp = df.loc[df['fhr']==pred]
#         #select columns
#         temp = temp[['lat','lon','vmax','mslp']]
#         temp = temp.add_prefix(str(model)+'_'+str(pred)+'_')
#         temp['datetime'] = pd.to_datetime(time, format = '%Y%m%d%H')
#         df_model = pd.concat([df_model, temp], axis=0)
#     df_out = df_out.merge(df_model, on='datetime', how='outer')
# df_out = df_out.sort_values(by='datetime') 

,SHIP_24_lat,SHIP_24_lon,SHIP_24_vmax,SHIP_24_mslp,datetime,AP01_24_lat,AP01_24_lon,AP01_24_vmax,AP01_24_mslp,AEMI_24_lat,...,HWRF_24_vmax,HWRF_24_mslp,NAM_24_lat,NAM_24_lon,NAM_24_vmax,NAM_24_mslp,CMC_24_lat,CMC_24_lon,CMC_24_vmax,CMC_24_mslp
0,17.1,-85.5,29.0,NaN,2018-10-05 18:00:00,18.5,-85.3,27.0,1007.0,NaN,...,NaN,NaN,16.9,-87.1,25.0,1006.0,NaN,NaN,NaN,NaN
1,18.0,-86.1,31.0,NaN,2018-10-06 00:00:00,NaN,NaN,NaN,NaN,17.8,...,NaN,NaN,NaN,NaN,NaN,NaN,18.3,-86.2,23.0,1005.0
2,17.9,-87.6,26.0,NaN,2018-10-06 06:00:00,17.7,-86.3,22.0,1006.0,19.0,...,37.0,1003.0,17.5,-86.9,27.0,1006.0,NaN,NaN,NaN,NaN
3,17.9,-87.3,27.0,NaN,2018-10-06 12:00:00,18.5,-86.7,30.0,1005.0,18.7,...,32.0,1005.0,NaN,NaN,NaN,NaN,19.3,-86.6,28.0,1002.0
4,19.8,-86.8,30.0,NaN,2018-10-06 18:00:00,NaN,NaN,NaN,NaN,20.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,20.8,-86.5,33.0,NaN,2018-10-07 00:00:00,20.3,-83.9,35.0,1001.0,NaN,...,43.0,999.0,19.8,-85.2,36.0,1001.0,20.1,-87.0,28.0,1002.0
6,21.2,-86.5,42.0,NaN,2018-10-07 06:00:00,19.5,-86.2,30.0,1001.0,21.0,...,42.0,997.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,21.7,-86.3,45.0,NaN,2018-10-07 12:00:00,21.1,-85.9,39.0,997.0,21.5,...,52.0,986.0,20.5,-85.6,46.0,996.0,21.2,-87.0,33.0,999.0
8,21.8,-86.2,61.0,NaN,2018-10-07 18:00:00,21.4,-85.7,43.0,994.0,21.4,...,68.0,978.0,21.0,-85.6,47.0,998.0,NaN,NaN,NaN,NaN
9,22.6,-85.6,74.0,NaN,2018-10-08 00:00:00,22.3,-85.8,50.0,990.0,22.8,...,86.0,965.0,22.0,-85.3,50.0,994.0,22.8,-86.4,42.0,994.0


In [6]:
df_out

,SHIP_24_lat,SHIP_24_lon,SHIP_24_vmax,SHIP_24_mslp,datetime,AP01_24_lat,AP01_24_lon,AP01_24_vmax,AP01_24_mslp,AEMI_24_lat,...,HWRF_24_vmax,HWRF_24_mslp,NAM_24_lat,NAM_24_lon,NAM_24_vmax,NAM_24_mslp,CMC_24_lat,CMC_24_lon,CMC_24_vmax,CMC_24_mslp
0,17.1,-85.5,29.0,NaN,2018-10-05 18:00:00,18.5,-85.3,27.0,1007.0,NaN,...,NaN,NaN,16.9,-87.1,25.0,1006.0,NaN,NaN,NaN,NaN
1,18.0,-86.1,31.0,NaN,2018-10-06 00:00:00,NaN,NaN,NaN,NaN,17.8,...,NaN,NaN,NaN,NaN,NaN,NaN,18.3,-86.2,23.0,1005.0
2,17.9,-87.6,26.0,NaN,2018-10-06 06:00:00,17.7,-86.3,22.0,1006.0,19.0,...,37.0,1003.0,17.5,-86.9,27.0,1006.0,NaN,NaN,NaN,NaN
3,17.9,-87.3,27.0,NaN,2018-10-06 12:00:00,18.5,-86.7,30.0,1005.0,18.7,...,32.0,1005.0,NaN,NaN,NaN,NaN,19.3,-86.6,28.0,1002.0
4,19.8,-86.8,30.0,NaN,2018-10-06 18:00:00,NaN,NaN,NaN,NaN,20.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,20.8,-86.5,33.0,NaN,2018-10-07 00:00:00,20.3,-83.9,35.0,1001.0,NaN,...,43.0,999.0,19.8,-85.2,36.0,1001.0,20.1,-87.0,28.0,1002.0
6,21.2,-86.5,42.0,NaN,2018-10-07 06:00:00,19.5,-86.2,30.0,1001.0,21.0,...,42.0,997.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,21.7,-86.3,45.0,NaN,2018-10-07 12:00:00,21.1,-85.9,39.0,997.0,21.5,...,52.0,986.0,20.5,-85.6,46.0,996.0,21.2,-87.0,33.0,999.0
8,21.8,-86.2,61.0,NaN,2018-10-07 18:00:00,21.4,-85.7,43.0,994.0,21.4,...,68.0,978.0,21.0,-85.6,47.0,998.0,NaN,NaN,NaN,NaN
9,22.6,-85.6,74.0,NaN,2018-10-08 00:00:00,22.3,-85.8,50.0,990.0,22.8,...,86.0,965.0,22.0,-85.3,50.0,994.0,22.8,-86.4,42.0,994.0


In [24]:
get_forecast(pali, 2016)

NameError: name 'pali' is not defined

In [30]:
# # for i in joined:
# #     out = pd.DataFrame({})
# #     out = pd.concat([out, joined[i]], axis=0)
# # join_forecast(df)

# df_all = pd.DataFrame()
# for i in range(len(joined)):
#     df_all = pd.concat([df_all, joined[i]], axis=0)
    
# df_all.shape

df_all

,SID,NAME,BASIN,ISO_TIME,LAT,LON,WMO_WIND,WMO_PRES,DIST2LAND,STORM_SPEED,...,AEMI_24_vmax,AEMI_24_mslp,AP01_24_lat,AP01_24_lon,AP01_24_vmax,AP01_24_mslp,CMC_24_lat,CMC_24_lon,CMC_24_vmax,CMC_24_mslp
0,2018217N12266,ileana,EP,2018-08-04 18:00:00,12.3000,-94.5000,25.0,1008.0,350.0,12.0,...,31.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2018217N12266,ileana,EP,2018-08-04 21:00:00,12.5828,-95.0197,25.0,1008.0,365.0,11.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2018217N12266,ileana,EP,2018-08-05 00:00:00,12.8000,-95.5000,25.0,1008.0,333.0,9.0,...,29.0,NaN,NaN,NaN,NaN,NaN,13.1,-97.9,34.0,1005.0
3,2018217N12266,ileana,EP,2018-08-05 03:00:00,12.9149,-95.9152,27.5,1007.0,311.0,8.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2018217N12266,ileana,EP,2018-08-05 06:00:00,13.0000,-96.3000,30.0,1006.0,292.0,8.0,...,37.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55,2018238N13239,miriam,EP,2018-09-01 09:00:00,21.2500,-141.0430,70.0,983.0,1450.0,11.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
56,2018238N13239,miriam,EP,2018-09-01 12:00:00,21.8000,-141.1000,65.0,986.0,1447.0,11.0,...,48.0,NaN,26.0,-144.9,28.0,1009.0,24.3,-144.8,25.0,1009.0
57,2018238N13239,miriam,EP,2018-09-01 15:00:00,22.3074,-141.1550,57.5,991.5,1446.0,10.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
58,2018238N13239,miriam,EP,2018-09-01 18:00:00,22.8000,-141.3000,50.0,997.0,1446.0,11.0,...,36.0,NaN,26.5,-144.6,28.0,1010.0,NaN,NaN,NaN,NaN


In [ ]:
#need to concat elements in this dictionary and merge columns of same name 

In [121]:
test = df1.loc[df1['NAME']=='hector'].copy()
test['fprecast']=0
df1.loc[df1['NAME']=='hector'] = test 
df1

,SID,NAME,BASIN,ISO_TIME,LAT,LON,WMO_WIND,WMO_PRES,DIST2LAND,STORM_SPEED,...,CARQ_24_vmax,CARQ_24_mslp,NAM_24_lat,NAM_24_lon,NAM_24_vmax,NAM_24_mslp,SHIP_24_lat,SHIP_24_lon,SHIP_24_vmax,SHIP_24_mslp
0,2018213N12245,hector,WP,2018-08-16 09:00:00,32.5545,165.646,35.0,1014.0,1979.0,6.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2018213N12245,hector,WP,2018-08-16 12:00:00,32.8500,165.610,35.0,1014.0,1953.0,6.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2018213N12245,hector,WP,2018-08-16 15:00:00,33.1926,165.707,35.0,1014.0,1932.0,8.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2018213N12245,hector,WP,2018-08-16 18:00:00,33.6000,165.833,35.0,1014.0,1903.0,8.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2018214N19153,shanshan,WP,2018-08-02 00:00:00,18.5000,152.800,35.0,1010.0,2228.0,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2995,2018292N14261,willa,EP,2018-10-21 03:00:00,15.7500,-106.107,62.5,991.5,393.0,6.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2996,2018292N14261,willa,EP,2018-10-21 06:00:00,16.0000,-106.300,70.0,987.0,391.0,6.0,...,NaN,NaN,NaN,NaN,NaN,NaN,17.6,-107.3,96.0,NaN
2997,2018292N14261,willa,EP,2018-10-21 09:00:00,16.2075,-106.458,77.5,981.0,380.0,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2998,2018292N14261,willa,EP,2018-10-21 12:00:00,16.4000,-106.600,85.0,975.0,366.0,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,18.1,-107.5,98.0,NaN


In [110]:
dic = df0.to_dict()
dic.update({'lala':1})

pd.DataFrame(dic)

,SID,NAME,BASIN,ISO_TIME,LAT,LON,WMO_WIND,WMO_PRES,DIST2LAND,STORM_SPEED,...,SIN_STORM_DIR,COS_LAT,SIN_LAT,COS_LON,SIN_LON,wind_category,YEAR,1_x,1_y,lala
0,2018213N12245,hector,WP,2018-08-16 09:00:00,32.5545,165.646,35.0,1014.0,1979.0,6.0,...,-0.258819,0.842880,0.538102,-0.968783,0.247912,1,2018,1.0,1.0,1
1,2018213N12245,hector,WP,2018-08-16 12:00:00,32.8500,165.610,35.0,1014.0,1953.0,6.0,...,0.087156,0.840094,0.542442,-0.968627,0.248521,1,2018,1.0,1.0,1
2,2018213N12245,hector,WP,2018-08-16 15:00:00,33.1926,165.707,35.0,1014.0,1932.0,8.0,...,0.241922,0.836835,0.547455,-0.969046,0.246881,1,2018,1.0,1.0,1
3,2018213N12245,hector,WP,2018-08-16 18:00:00,33.6000,165.833,35.0,1014.0,1903.0,8.0,...,0.241922,0.832921,0.553392,-0.969586,0.244749,1,2018,1.0,1.0,1


In [55]:
storm = hurdat.get_storm(('hector', 2018))
forecast = storm.get_operational_forecasts()


In [70]:
forecast['AEMN']

{'2018072806': {'fhr': [0,
   6,
   12,
   18,
   24,
   30,
   36,
   42,
   48,
   54,
   60,
   66,
   72,
   78,
   84,
   90,
   96,
   102,
   108,
   114,
   120,
   126,
   132,
   138,
   144,
   150,
   156,
   162,
   168,
   174,
   180,
   186,
   192,
   198,
   204,
   210,
   216,
   222,
   228,
   234,
   240],
  'lat': [7.5,
   7.4,
   7.800000000000001,
   8.200000000000001,
   8.5,
   8.700000000000001,
   8.9,
   9.0,
   9.1,
   9.4,
   9.5,
   9.700000000000001,
   10.0,
   10.200000000000001,
   10.4,
   10.5,
   10.700000000000001,
   11.0,
   11.200000000000001,
   11.4,
   11.5,
   11.700000000000001,
   11.9,
   12.100000000000001,
   12.3,
   12.600000000000001,
   12.8,
   13.0,
   13.200000000000001,
   13.4,
   13.5,
   13.8,
   13.9,
   14.100000000000001,
   14.200000000000001,
   14.4,
   14.600000000000001,
   14.8,
   15.0,
   15.200000000000001,
   15.200000000000001],
  'lon': [-97.7,
   -98.9,
   -99.80000000000001,
   -101.2,
   -102.60000000000

In [27]:
df0[['NAME','SID']][-10000:]

,NAME,SID
257588,hector,2018213N12245
257589,hector,2018213N12245
257590,hector,2018213N12245
257591,hector,2018213N12245
257592,shanshan,2018214N19153
...,...,...
260583,willa,2018292N14261
260584,willa,2018292N14261
260585,willa,2018292N14261
260586,willa,2018292N14261
